In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import time,os
import lightgbm as lgb
from tqdm import tqdm
from geohash import encode
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import ParameterGrid
from sklearn.feature_selection import VarianceThreshold

%config InlineBackend.figure_format = 'retina' # set 'png' here when working on notebook
%matplotlib inline

## 加载数据

In [2]:
# 将wifi_infos转成dict
def make_wifi_dict(df):
    def _split(x):
        x=x.split('|')[:2]
        x[1]=int(x[1])
        return x
    df.wifi_infos=df.wifi_infos.apply(lambda x:dict([_split(i) for i in x.split(';')]))
    return df

# 提取时间结构信息
def make_struct_time(df):
    time_infos=[]
    for t in tqdm(df.time_stamp):
        _struct_time=time.strptime(t,'%Y-%m-%d %H:%M')
        time_infos.append([_struct_time.tm_hour+_struct_time.tm_min/60.,_struct_time.tm_hour,_struct_time.tm_wday])
    time_infos=pd.DataFrame(time_infos,columns=['time','hour','wday'])
    df=pd.concat((df,time_infos),axis=1)
    return df

def load_data(file_path):
    result=pd.read_csv(file_path)
    result=make_wifi_dict(result)
    result=make_struct_time(result)
    return result

meta_df=pd.read_csv('data/训练数据-ccf_first_round_shop_info.csv')
train_df=load_data('data/训练数据-ccf_first_round_user_shop_behavior.csv')
test_df=load_data('data/AB榜测试集-evaluation_public.csv')
# 加入商店信息
train_df=pd.merge(train_df,meta_df[['shop_id','mall_id']],on='shop_id',how='left')

100%|██████████| 483931/483931 [00:04<00:00, 100900.53it/s]


In [3]:
x_train=train_df

In [3]:
# 构造线下验证集
x_train=train_df[train_df.time_stamp<'2017-08-30 00:00']
x_val=train_df[train_df.time_stamp>='2017-08-30 00:00']

In [4]:
x_train.head()

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos,time,hour,wday,mall_id
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,"{'b_5857370': -68, 'b_56326644': -89, 'b_28723...",21.333333,21,6,m_1409
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,"{'b_6396479': -57, 'b_5857370': -68, 'b_563281...",21.333333,21,6,m_1409
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,"{'b_8006442': -71, 'b_33503892': -56, 'b_80063...",13.166667,13,2,m_4079
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,"{'b_26250581': -64, 'b_52934996': -79, 'b_2625...",12.500000,12,6,m_6587
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,"{'b_54466444': -86, 'b_21685901': -91, 'b_3900...",19.833333,19,4,m_3005


## 添加特征

In [5]:
# 计算两点之间距离
def cal_distance(lat1, lon1, lat2, lon2):
    dx = np.abs(lon1 - lon2)  # 经度差
    dy = np.abs(lat1 - lat2)  # 维度差
    b = (lat1 + lat2) / 2.0
    Lx = 6371004.0 * (dx / 57.2958) * np.cos(b / 57.2958)
    Ly = 6371004.0 * (dy / 57.2958)
    L = (Lx ** 2 + Ly ** 2) ** 0.5
    return L

# 计算wifi稀疏矩阵
def add_one_hot_wifi(df1,df2):
    vec = DictVectorizer()
    vec.fit(df1.wifi_infos)
    result=vec.transform(df2.wifi_infos).toarray()
    return result

# 计算geo稀疏矩阵/定位经纬度
def add_one_hot_geo(df):
    return df[['longitude','latitude']].values

# 增加时间特征
def add_time(df):
    return df[['time','wday']].values

def add_feats(df1,df2):
    feats=[]
    feats.append(add_one_hot_wifi(df1,df2))
    feats.append(add_one_hot_geo(df2))
#     feats.append(add_time(df2))
#     feats=np.concatenate(feats,axis=1)
    return feats

## 模型训练

In [6]:
result=[]
precision=[]

# for mall_id in tqdm(['m_7800','m_690','m_7168','m_6337','m_1377']):
for mall_id in tqdm(meta_df.mall_id.unique()):
    _x_train=x_train[x_train.mall_id==mall_id]
#     _x_val=x_val[x_val.mall_id==mall_id] #
    _test_df=test_df[test_df.mall_id==mall_id]
    _meta_df=meta_df[meta_df.mall_id==mall_id]
    # 构造特征
    train_feat=add_feats(_x_train,_x_train)
#     val_feat=add_feats(_x_train,_x_val) #
    test_feat=add_feats(_x_train,_test_df)
    # 降维
    nb_wifi_column=train_feat[0].shape[1]
    sel = VarianceThreshold(threshold=1)
    train_feat[0]=sel.fit_transform(train_feat[0])
#     val_feat[0]=sel.transform(val_feat[0]) #
    test_feat[0]=sel.transform(test_feat[0])
    train_feat=np.concatenate(train_feat,axis=1)
#     val_feat=np.concatenate(val_feat,axis=1) #
    test_feat=np.concatenate(test_feat,axis=1)
    print('wifi nb_columns:{}->{}'.format(nb_wifi_column,train_feat.shape[1]))
    # 添加标签
    encoder=LabelEncoder()
    encoder.fit(_meta_df.shop_id)
    train_label=encoder.transform(_x_train.shop_id)
#     val_label=encoder.transform(_x_val.shop_id) #
    # 训练
    nb_class=_meta_df.shape[0]
    params = {
        'num_class':[nb_class],
        'objective': ['multiclass'],
        'metric':['multi_error'],
        'learning_rate':[0.05],
        'feature_fraction': [0.6],
        'max_depth': [12],# 14
        'num_leaves':[180],# 256
        'bagging_fraction': [0.6],
        'bagging_freq':[5],
        'min_data_in_leaf':[10],
        'min_gain_to_split':[0],
        'lambda_l1':[1],
        'lambda_l2':[1],
        'verbose':[0],
        'is_unbalance':[True]
    }
    params=list(ParameterGrid(params))
    lgb_train=lgb.Dataset(train_feat,train_label)
    model = lgb.train(params[0], lgb_train, num_boost_round=150)
    # 模型验证
#     proba=model.predict(val_feat)
#     predict_label=np.argmax(proba,axis=1)
#     ans=pd.DataFrame({'pred':predict_label,'true':val_label})
#     score={mall_id:ans[ans.pred==ans.true].shape[0]/ans.shape[0]}
#     precision.append(score)
#     print(score)
    # 预测
#     proba=model.predict(test_feat)
#     predict_label=np.argmax(proba,axis=1)
#     ans=pd.DataFrame({'row_id':_test_df.row_id,'shop_id':encoder.inverse_transform(predict_label)})
#     result.append(ans)
    # 保存候选集
    nb_candidates=7
    proba=model.predict(train_feat)
    candidates=encoder.inverse_transform(proba.argsort(axis=1)[:,-nb_candidates:])
    ans=pd.DataFrame(candidates,index=_x_train.index)
    ans.to_hdf('cache/multi/{}_train_full.hdf'.format(mall_id),'data')
    proba.sort(axis=1)
    ans=pd.DataFrame(proba[:,-nb_candidates:],index=_x_train.index)
    ans.to_hdf('cache/multi/{}_train_full.hdf'.format(mall_id),'proba')
    ###
#     proba=model.predict(val_feat)
#     candidates=encoder.inverse_transform(proba.argsort(axis=1)[:,-nb_candidates:])
#     ans=pd.DataFrame(candidates,index=_x_val.index)
#     ans.to_hdf('cache/multi/{}_val.hdf'.format(mall_id),'data')
#     proba.sort(axis=1)
#     ans=pd.DataFrame(proba[:,-nb_candidates:],index=_x_val.index)
#     ans.to_hdf('cache/multi/{}_val.hdf'.format(mall_id),'proba')
    ###
    proba=model.predict(test_feat)
    candidates=encoder.inverse_transform(proba.argsort(axis=1)[:,-nb_candidates:])
    ans=pd.DataFrame(candidates,index=_test_df.row_id)
    ans.to_hdf('cache/multi/{}_test.hdf'.format(mall_id),'data')
    proba.sort(axis=1)
    ans=pd.DataFrame(proba[:,-nb_candidates:],index=_test_df.row_id)
    ans.to_hdf('cache/multi/{}_test.hdf'.format(mall_id),'proba')

  0%|          | 0/97 [00:00<?, ?it/s]

wifi nb_columns:13294->1772


  1%|          | 1/97 [00:44<1:10:39, 44.16s/it]

wifi nb_columns:2701->1475


  2%|▏         | 2/97 [00:59<56:27, 35.65s/it]  

...

 98%|█████████▊| 95/97 [28:24<00:22, 11.23s/it]

wifi nb_columns:4779->1648


 99%|█████████▉| 96/97 [28:40<00:12, 12.50s/it]

wifi nb_columns:3654->1530


100%|██████████| 97/97 [29:15<00:00, 19.35s/it]


In [8]:
# 线下验证
tmp=pd.concat([pd.Series(i) for i in precision])
tmp=pd.DataFrame({'mall_id':tmp.index,'precision':tmp.values})
t2=x_val[['mall_id','shop_id']].groupby('mall_id',as_index=False)['shop_id'].agg({'count':'count'})
t=pd.merge(tmp,t2,on='mall_id',how='inner')
sum(t.precision*t['count'])/sum(t['count']) 
#150：0.90364919322725024
#early stopping：0.90777464519520235

0.90463675315780256

In [62]:
# 保存结果
tmp=pd.concat(result)
print(tmp.shape)
tmp.to_csv('result.csv',index=False,sep=',')

(483931, 2)


ideas:

1.early_stopping多分类能否提高（貌似没有改善）

2.增大候选集到7个（貌似没有改善）/减少到3个

3.用户历史

4.二分类加入多分类概率信息

## EDA
1.很多错误答案和正确答案的shop经纬度一样

2.很多错误答案的置信度分布并不极端，分类器并不自信

In [138]:
train_proba=model.predict(train_feat)
val_proba=model.predict(val_feat)

In [140]:
pd.DataFrame(encoder.inverse_transform(val_proba.argsort(axis=1)[:,-5:]))

,0,1,2,3,4
0,s_679110,s_479529,s_712117,s_521874,s_472478
1,s_685227,s_992598,s_683217,s_682074,s_684555
2,s_682074,s_683674,s_683671,s_683053,s_684547
3,s_698556,s_473743,s_539726,s_681760,s_685058
4,s_809873,s_539726,s_3382317,s_696563,s_461325
5,s_539726,s_683832,s_1462898,s_685058,s_1114306
6,s_679110,s_683678,s_685058,s_466068,s_774995
7,s_683671,s_683821,s_557771,s_2248290,s_635423
8,s_3382317,s_809873,s_683217,s_696563,s_461325
9,s_683985,s_465792,s_698556,s_539726,s_685058


In [146]:
val_proba.sort(axis=1)
pd.DataFrame(val_proba[:,-5:])

,0,1,2,3,4
0,0.003684,0.004195,0.004405,0.009055,0.950170
1,0.020446,0.031947,0.125492,0.317416,0.410895
2,0.063578,0.069018,0.123614,0.140901,0.298582
3,0.007020,0.007237,0.007717,0.013129,0.887729
4,0.000812,0.001812,0.002056,0.002456,0.975247
5,0.000688,0.000830,0.000881,0.001080,0.987465
6,0.000191,0.000213,0.000249,0.000264,0.993952
7,0.000455,0.000478,0.000830,0.005798,0.983089
8,0.004922,0.007463,0.007641,0.007736,0.922186
9,0.000608,0.001486,0.002664,0.011097,0.976144
